Defining an "adopted user" as a user who has logged into the product on three separate
days in at least one seven­day period , identify which factors predict future user
adoption .


Please send us a brief writeup of your findings (the more concise, the better ­­ no more
than one page), along with any summary tables, graphs, code, or queries that can help
us understand your approach.


Please note any factors you considered or investigation
you did, even if they did not pan out. Feel free to identify any further research or data
you think would be valuable.

## Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Read Data

In [ ]:
df1 = pd.read_csv('./takehome_users.csv')
df2 = pd.read_csv('./takehome_user_engagement.csv')

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df = df1.merge(df2, how='left', on='user')